In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, accuracy_score
import xgboost as xgb
import pickle

In [2]:
class ModelTrainer:
    def __init__(self, data_path, target_column, numeric_columns, categorical_columns, model_type='RandomForest'):
        self.data_path = data_path
        self.target_column = target_column
        self.numeric_columns = numeric_columns
        self.categorical_columns = categorical_columns
        self.model_type = model_type
        self.model = None
        self.pipeline = None

    def load_data(self):
        self.data = pd.read_csv(self.data_path)
        self.X = self.data.drop(self.target_column, axis=1)
        self.y = self.data[self.target_column]

    def setup_pipeline(self):
        numeric_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),  # Menggunakan median untuk imputasi
            ('scaler', StandardScaler())
        ])
        categorical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),  # Menggunakan modus untuk imputasi
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ])
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numeric_transformer, self.numeric_columns),
                ('cat', categorical_transformer, self.categorical_columns)
            ])

        if self.model_type == 'RandomForest':
            self.model = RandomForestClassifier(random_state=42)
        elif self.model_type == 'XGBoost':
            self.model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
        else:
            raise ValueError("Unsupported model type")

        self.pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', self.model)
        ])

    def train_model(self):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.2, random_state=42)
        self.pipeline.fit(self.X_train, self.y_train)

    def evaluate_model(self):
        predictions = self.pipeline.predict(self.X_test)
        print(classification_report(self.y_test, predictions))
        print("Accuracy: ", accuracy_score(self.y_test, predictions))

    def save_model(self, file_path):
        with open(file_path, 'wb') as f:
            pickle.dump(self.pipeline, f)

In [4]:
if __name__ == "__main__":
    numeric_cols = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
    categorical_cols = ['Geography', 'Gender']
    trainer = ModelTrainer(data_path='/content/data_D.csv', target_column='churn', numeric_columns=numeric_cols, categorical_columns=categorical_cols, model_type='RandomForest')
    trainer.load_data()
    trainer.setup_pipeline()
    trainer.train_model()
    trainer.evaluate_model()
    trainer.save_model('/content/best_data (2).pkl')

              precision    recall  f1-score   support

           0       0.88      0.95      0.91      6513
           1       0.72      0.50      0.59      1739

    accuracy                           0.85      8252
   macro avg       0.80      0.73      0.75      8252
weighted avg       0.84      0.85      0.84      8252

Accuracy:  0.8537324285021813
